In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.subplots as sp
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
import json
from datetime import datetime
import plotly.express as px
import tensorflow as tfs

from astral import LocationInfo
from astral.sun import sun
import datetime

import plotly.graph_objects as go

In [ ]:
#connect to database
bucket = "rp_one_m"

client = InfluxDBClient(url="http://localhost:8086", token="6nJJG2EUP8hGm0gsbi-TXScDIiMCMl03TMleXQ2gr4m9l5EO1XMGa97D0tmUebQyGOEo_fR_vsbwZrNpucVcMQ==", org="thesis")

write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

In [ ]:
#get the id for the database
#7847f5f7
#45b46fef
#15ecc075
#ee9f3d22
baseId = "7847f5f7"
battery_id = baseId + "-battery"
inverter_id = baseId + "-inverter"
grid_id = baseId + "-grid"
battery2grid_id = baseId + "-battery2grid"
    
print(battery_id)

In [ ]:
#Query the database for the data

#query for solarInverterMetrics
tables_inverter = query_api.query(f'''
from(bucket: "rp_one_m")
  |> range(start: 2018-01-01T00:00:01Z, stop: 2024-11-18T23:30:00Z)
  |> filter(fn: (r) => r["_measurement"] == "solarInverterMetrics")
  |> filter(fn: (r) => r["nodeId"] == "{inverter_id}")
  |> pivot(
      rowKey:["_time"], 
      columnKey:["_field"], 
      valueColumn:"_value"
  )
  |> sort(columns: ["_time"])  //sort by timestamp''')

#query for submeteringMetrics
tables_submetering = query_api.query(f'''
from(bucket: "rp_one_m")
  |> range(start: 2018-01-01T00:00:01Z, stop: 2024-11-18T23:30:00Z)
  |> filter(fn: (r) => r["_measurement"] == "submeteringMetrics")
  |> filter(fn: (r) => r["nodeId"] == "{grid_id}")
  |> pivot(
      rowKey:["_time"], 
      columnKey:["_field"], 
      valueColumn:"_value"
  )
  |> sort(columns: ["_time"])  //sort by timestamp''')

#query for batteryMetrics
tables_battery = query_api.query(f'''
from(bucket: "rp_one_m")
  |> range(start: 2018-01-01T00:00:01Z, stop: 2024-11-18T23:30:00Z)
  |> filter(fn: (r) => r["_measurement"] == "batteryMetrics")
  |> filter(fn: (r) => r["nodeId"] == "{battery_id}")
  |> pivot(
      rowKey:["_time"], 
      columnKey:["_field"], 
      valueColumn:"_value"
  )
  |> sort(columns: ["_time"])  //sort by timestamp''')

#query for battery2grid
tables_battery2grid = query_api.query(f'''
from(bucket: "rp_one_m")
  |> range(start: 2018-01-01T00:00:01Z, stop: 2024-11-18T23:30:00Z)
  |> filter(fn: (r) => r["_measurement"] == "submeteringMetrics")
  |> filter(fn: (r) => r["nodeId"] == "{battery2grid_id}")
  |> pivot(
      rowKey:["_time"], 
      columnKey:["_field"], 
      valueColumn:"_value"
  )
  |> sort(columns: ["_time"])  //sort by timestamp''')

#convert query results to DataFrame for each measurement
data_inverter = []
data_submetering = []
data_battery = []
data_battery2grid = []

#process inverter data
for table in tables_inverter:
    for row in table.records:
        data_inverter.append(row.values)
df_inverter = pd.DataFrame(data_inverter)

#process submetering data
for table in tables_submetering:
    for row in table.records:
        data_submetering.append(row.values)
df_submetering = pd.DataFrame(data_submetering)

#process battery data
for table in tables_battery:
    for row in table.records:
        data_battery.append(row.values)
df_battery = pd.DataFrame(data_battery)

#process battery2grid data
for table in tables_battery2grid:
    for row in table.records:
        data_battery2grid.append(row.values)
df_battery2grid = pd.DataFrame(data_battery2grid)

In [ ]:
#give subscripts
df_inverter = df_inverter.rename(columns=lambda x: x + '_inverter')
df_submetering = df_submetering.rename(columns=lambda x: x + '_submetering')
df_battery = df_battery.rename(columns=lambda x: x + '_battery')
df_battery2grid = df_battery2grid.rename(columns=lambda x: x + '_battery2grid')
#put _time back to normal
df_inverter.rename(columns={'_time_inverter':'_time'}, inplace=True)
df_submetering.rename(columns={'_time_submetering':'_time'}, inplace=True)
df_battery.rename(columns={'_time_battery':'_time'}, inplace=True)
df_battery2grid.rename(columns={'_time_battery2grid':'_time'}, inplace=True)

In [ ]:
#combine all dataframes on _time column
#indicator: Adds a column to the output dataFrame called _merge that shows which dataFrame each row originated from.
df_combined = pd.merge(df_inverter, df_submetering, on='_time', how='outer', suffixes=('_inverter', '_submetering'), indicator='Origin_1_2')
df_combined = pd.merge(df_combined, df_battery, on='_time', how='outer', suffixes=('', '_battery'), indicator='Origin_1_2_3')
df_combined = pd.merge(df_combined, df_battery2grid, on='_time', how='outer', suffixes=('', '_battery2grid'), indicator='Origin_1_2_3_4')

#check if all have same time columns
if(df_combined['Origin_1_2'].equals(df_combined['Origin_1_2_3']) and df_combined['Origin_1_2_3'].equals(df_combined['Origin_1_2_3_4'])):
    print("all have same timestamps")
    df_combined.drop(columns=['Origin_1_2', 'Origin_1_2_3', 'Origin_1_2_3_4'], inplace=True)
else:
    print("not all have same timestamps")

#drop columns with suffixes for _start and _stop
columns_to_drop = [col for col in df_combined.columns if col.startswith('_start') or col.startswith('_stop')]
df_combined.drop(columns=columns_to_drop, inplace=True)

#keep only one _start and _stop column from the first dataframe (inverter)
df_combined['_start'] = df_inverter['_start_inverter']
df_combined['_stop'] = df_inverter['_stop_inverter']

#display the first 10 rows of the combined dataframe
pd.set_option('display.max_columns', None)

df_combined.head(10)

In [ ]:
#remove all unnecessary
nodeID = df_combined['nodeId_battery'][0]
startTime = df_combined['_time'][0]
endTime = df_combined['_time'].iloc[-1]
df_combined.drop(columns=['result_inverter', 'nodeId_inverter', 'result_submetering', 'nodeId_submetering', 'result_battery',
                          '_measurement_battery', 'result_battery2grid', '_measurement_battery2grid', 'nodeId_battery2grid',
                         'table_inverter', 'table_submetering', 'table_battery', 'table_battery2grid',
                         '_measurement_inverter', '_measurement_submetering', 'nodeId_battery'], inplace=True)
#drop deltas, not relevant since meeting 27112024
df_combined.drop(columns=['sum_producedEnergyDeltaTot_Wh_inverter', 'sum_exportedEnergyDeltaTot_Wh_submetering',
                         'sum_importedEnergyDeltaTot_Wh_submetering', 'sum_chargedEnergyDeltaTot_Wh_battery',
                         'sum_dischargedEnergyDeltaTot_Wh_battery', 'sum_exportedEnergyDeltaTot_Wh_battery2grid',
                         'sum_importedEnergyDeltaTot_Wh_battery2grid'], inplace=True)
df_combined.drop(columns=['_start', '_stop'], inplace=True)
df_combined

# drop bad data

In [ ]:
initial = go.Figure()

initial.add_trace(go.Scatter(x=df_combined["_time"], y=df_combined["mean_actualPowerTot_W_inverter"], mode='lines', name='transformation 2'))

# Update layout for better visualization
initial.update_layout(
    title='initial data',
    xaxis_title='Time',
    yaxis_title='Mean Actual Power (W)',
    xaxis_rangeslider_visible=True
)

# Show the plot
initial.show()

# ee9f3d22
df_combined.drop(df_combined[df_combined['_time'] <= "2021-05-12 00:00:00+00:00"].index, inplace = True)
df_inverterPower = df_combined[['_time', 'mean_actualPowerTot_W_inverter']]
df_inverterPower

# 15ecc075
df_combined.drop(df_combined[df_combined['_time'] <= "2021-08-12 00:00:00+00:00"].index, inplace = True)
df_inverterPower = df_combined[['_time', 'mean_actualPowerTot_W_inverter']]
df_inverterPower

# 45b46fef
df_combined.drop(df_combined[df_combined['_time'] <= "2021-05-5 00:00:00+00:00"].index, inplace = True)
df_inverterPower = df_combined[['_time', 'mean_actualPowerTot_W_inverter']]
df_inverterPower

In [ ]:
# 7847f5f7
df_combined.drop(df_combined[(df_combined['_time'] >= '2020-01-21 06:20:00+00:00') & (df_combined['_time'] < '2021-02-10 06:24:00+00:00')].index,
                inplace = True)

df_combined.drop(df_combined[df_combined['_time'] < '2021-02-11 11:32:00+00:00'].index, inplace = True)

# 0033d164
#df_combined.drop(df_combined[df_combined['_time'] < '2021-09-17 00:00:00+00:00'].index, inplace = True)

df_inverterPower = df_combined[['_time', 'mean_actualPowerTot_W_inverter']]
df_inverterPower

In [ ]:
#read in the households file with lats and longs + only the ids that are in the database
#json files
file_path = r"C:\Users\samr0\OneDrive - KU Leuven\Documents\!School\master\Thesis\data\households_in_database.json"
#read JSON into a dataFrame
df_households = pd.read_json(file_path)

df_households.head()

In [ ]:
city = df_households[df_households['id'] == baseId]['city'].values[0]
print(city)

In [ ]:
#create a new DataFrame with a complete range of 1-minute intervals
full_time_range = pd.date_range(start=df_inverterPower['_time'].min(), end=df_inverterPower['_time'].max(), freq='1min')
df_full_range = pd.DataFrame(full_time_range, columns=['_time'])

#merge the original DataFrame with the full range DataFrame
df_merged = pd.merge(df_full_range, df_inverterPower, on='_time', how='left')

#convert the DataFrame to appropriate dtypes
df_merged = df_merged.infer_objects()

#for interpolation make sure that production is zero when sun is not up
df_merged.set_index('_time', inplace=True)
city = LocationInfo(city, "Belgium")

In [ ]:
import pytz #ensure timezone consistency
def get_sunrise_sunset(date):
    s = sun(city.observer, date=date)
    return s['sunrise'].astimezone(pytz.timezone(city.timezone)), s['sunset'].astimezone(pytz.timezone(city.timezone))

df_merged['date'] = df_merged.index.date
sun_times = {date: get_sunrise_sunset(date) for date in df_merged['date'].unique()}

#vectorized operation: fix inconsistencies in dataframe due to resampling
for date, (sunrise, sunset) in sun_times.items():
    #from midnight to sunrise
    mask_before_sunrise = (df_merged.index >= pd.Timestamp(date, tz='Europe/Brussels')) & (df_merged.index < sunrise)
    #from sunset till midnight
    mask_after_sunset = (df_merged.index > sunset) & (df_merged.index < pd.Timestamp(date, tz='Europe/Brussels') + pd.Timedelta(days=1))
    
    df_merged.loc[mask_before_sunrise, 'mean_actualPowerTot_W_inverter'] = 0
    df_merged.loc[mask_after_sunset, 'mean_actualPowerTot_W_inverter'] = 0

df_merged.drop(columns=['date'], inplace=True)

print(df_merged)

In [ ]:
#interpolate the missing values
df_inverterPower_interpolated = df_merged.interpolate(method='linear')
df_inverterPower_interpolated['mean_actualPowerTot_W_inverter'] = df_inverterPower_interpolated['mean_actualPowerTot_W_inverter'].round(1)

#print the interpolated DataFrame
print("Interpolated DataFrame:")
df_inverterPower_interpolated.info()

In [ ]:
df_data = df_inverterPower_interpolated.copy()
df_data

file_path = "C:\Users\samr0\OneDrive - KU Leuven\Documents\!School\master\Thesis\data\inverter_power_data_7847f5f7.csv"

df_data.to_csv(file_path)
#df_data.to_csv('inverter_power_data_0033d164.csv')

#comment when running whole file

#7847f5f7
file_path = r"C:\Users\samr0\OneDrive - KU Leuven\Documents\!School\master\Thesis\data\inverter_power_data_7847f5f7.csv"

df_data = pd.read_csv(file_path, index_col='_time', parse_dates=True)

#df_data = pd.read_csv('inverter_power_data_0033d164.csv', index_col='_time', parse_dates=True)

df_data.index = df_data.index.tz_convert('Europe/Brussels')

df_data

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_data.index, y=df_data['mean_actualPowerTot_W_inverter'], mode='lines', name='Power'))

# Update layout for better visualization
fig.update_layout(
    title='Inverter Power Over Time',
    xaxis_title='Time',
    yaxis_title='Mean Actual Power (W)',
    #xaxis_rangeslider_visible=True,
    margin=dict(t=150),  # Increase top margin to fit legend and title
    legend=dict(
        orientation="h",  # horizontal layout
        y=1,           # place it above the plot area
        x=0.5,
        xanchor='center',
        yanchor='bottom'
    ),
    plot_bgcolor='white',
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        range=['2021-05-07 00:00:00', '2021-05-07 23:59:59']
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgray'
    )
)

# Show the plot
fig.show()

# drop the beginning where there is no data


#7847f5f7

df_data = df_data[df_data.index >= '2021-03-11']
df_data

# downsample to 15 min

In [ ]:
#downsampling needs to be before the normalisation

df_data = df_data.resample(rule = '15min', closed='left', label='right').mean()
df_data = df_data.round()

nan_rows = df_data[df_data['mean_actualPowerTot_W_inverter'].isna()]
print(nan_rows)

In [ ]:
df_data

In [ ]:
df_data[df_data["mean_actualPowerTot_W_inverter"] > 0]

In [ ]:
df_data.index = pd.to_datetime(df_data.index)

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_data.index, y=df_data['mean_actualPowerTot_W_inverter'], mode='lines', name='Power'))

#update layout for better visualization
fig.update_layout(
    title='Inverter Power Over Time',
    xaxis_title='Time',
    yaxis_title='Mean Actual Power (W)',
    xaxis_rangeslider_visible=True
)

fig.show()

In [ ]:
def remove_outliers_iqr_day(series, k=1.5):
    """
    Compute Q1, Q3, and IQR for the given series and replace values outside
    [Q1 - k*IQR, Q3 + k*IQR] with NaN.
    """
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - k * IQR
    upper_bound = Q3 + k * IQR
    return series.where((series >= lower_bound) & (series <= upper_bound), np.nan)

def fill_outliers_with_moving_avg(series, window=5):
    """
    Replace NaN values in the series (i.e. outliers) with the moving average
    of the previous 'window' timesteps. (Uses shift(1) to ensure only previous values are used.)
    """
    rolling_avg = series.shift(1).rolling(window=window, min_periods=1).mean()
    return series.fillna(rolling_avg)

def process_day_production(series, k=1.5, window=5):
    """
    For a day's series:
      - Identify the production period: from the first nonzero to the last nonzero value.
      - Apply IQR outlier filtering to that period.
      - Replace outliers (NaNs) with the moving average of the previous 'window' values.
    The rest of the day (typically night when production is 0) remains unchanged.
    """
    #identify production time where production > 0
    production_mask = series > 0
    if production_mask.sum() == 0:
        #if there is no production at all, return the series as is.
        return series

    #get the timestamps where production occurs
    production_times = series[production_mask].index
    first_time = production_times[0]
    last_time = production_times[-1]
    
    #extract the production period
    production_series = series.loc[first_time:last_time]
    
    #remove outliers using IQR for this subset
    production_clean = remove_outliers_iqr_day(production_series, k=k)
    
    #replace the outliers with a moving average of the previous 'window' values
    production_filled = fill_outliers_with_moving_avg(production_clean, window=window)
    
    #update the original series for this day with the cleaned production period
    series.loc[first_time:last_time] = production_filled
    return series

#now apply this function day-by-day
df_filtered_day = df_data.copy()

df_filtered_day['mean_actualPowerTot_W_inverter'] = df_filtered_day.groupby(
pd.Grouper(freq='D')
)['mean_actualPowerTot_W_inverter'].transform(
    lambda s: process_day_production(s, k=1.5, window=5)
)

#if there are nan's at the beginning of the day
df_filtered_day['mean_actualPowerTot_W_inverter'] = df_filtered_day['mean_actualPowerTot_W_inverter'].ffill()

df_filtered_day

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_data.index, y=df_data['mean_actualPowerTot_W_inverter'], mode='lines', name='Power_original'))
fig.add_trace(go.Scatter(x=df_filtered_day.index, y=df_filtered_day['mean_actualPowerTot_W_inverter'], mode='lines', name='Power'))

#update layout for better visualization
fig.update_layout(
    title='Inverter Power Over Time',
    xaxis_title='Time',
    yaxis_title='Mean Actual Power (W)',
    xaxis_rangeslider_visible=True
)

#show the plot
fig.show()

In [ ]:
df_data = df_filtered_day

In [ ]:
# Pmax(t) profile
df_data.index = pd.to_datetime(df_data.index)

#extract the time of day
df_data['time_of_day'] = df_data.index.time

# group by the time of day and calculate the maximum value
#max_profile = df_data.groupby('time_of_day')['mean_actualPowerTot_W_inverter'].quantile(0.95)
max_profile = df_data.groupby('time_of_day')['mean_actualPowerTot_W_inverter'].max()
df_data = df_data.drop(columns=['time_of_day'])
max_profile

In [ ]:
df_max_profile = pd.DataFrame(max_profile)
df_max_profile

In [ ]:
fig3 = go.Figure()

fig3.add_trace(go.Scatter(x=max_profile.index, y=max_profile.values, mode='lines', name='Power'))

#update layout for better visualization
fig3.update_layout(
    title='max profile',
    xaxis_title='Time',
    yaxis_title='Mean Actual Power (W)',
    xaxis_rangeslider_visible=True
)

fig3.show()

In [ ]:
# transformation 1 for Pmax: Savitzky-golay filter

from scipy.signal import savgol_filter

#The Savitzky-Golay filter applies a least-squares polynomial fit over a moving window.
#To maintain symmetry, the filter must have an equal number of points on both sides of the center point.
window_size = 31 #41
poly_order = 3 #3
smoothed_profile = savgol_filter(max_profile, window_length=window_size, polyorder=poly_order, mode='nearest')

#convert back to a Pandas Series
smoothed_profile = pd.Series(smoothed_profile, index=max_profile.index)

#set any values below zero to zero
smoothed_profile[smoothed_profile < 0] = 0

step1 = go.Figure()

step1.add_trace(go.Scatter(x=max_profile.index, y=max_profile.values, mode='lines', name='original'))
step1.add_trace(go.Scatter(x=smoothed_profile.index, y=smoothed_profile.values, mode='lines', name='step1'))

#update layout for better visualization
step1.update_layout(
    title='max profile step1',
    xaxis_title='Time',
    yaxis_title='Mean Actual Power (W)',
    xaxis_rangeslider_visible=True
)

step1.show()

In [ ]:
# transformation 2: adjust for seasonal variations

#In summer, the daylight hours are longer, so the power curve should be stretched.
#In winter, the daylight hours are shorter, so the power curve should be squeezed.

#use first and last non-zero production times as sunrise and sunset
import pytz #ensure timezone consistency
city = LocationInfo("Brussels", "Belgium")

#ensure datetime index
df_data.index = pd.to_datetime(df_data.index)

#group by each day and find T_start and T_end separately
#changed to use the actual sunrise and sunset to prevent weird behavior on cloudy days
def find_sun_times(day):
    date = day.index[0].date()
    s = sun(city.observer, date=date)
    return pd.Series({
        "T_start": s['sunrise'].astimezone(pytz.timezone(city.timezone)).time().replace(microsecond = 0),  #first nonzero time
        "T_end": s['sunset'].astimezone(pytz.timezone(city.timezone)).time().replace(microsecond = 0)  #last nonzero time
    })

#apply function per day
daily_sun_times = df_data.groupby(df_data.index.date).apply(find_sun_times)

#reset index to keep dates
daily_sun_times.reset_index(inplace=True)

print(daily_sun_times)

In [ ]:
# next in transformation 2, stretch and squeze

#convert smoothed_profile to dataFrame for easy manipulation
df_smoothed_profile = smoothed_profile.reset_index()
df_smoothed_profile.columns = ["time_of_day", "P_max"]

#convert time_of_day to minutes since midnight for easier calculations
df_smoothed_profile["minutes_since_midnight"] = df_smoothed_profile["time_of_day"].apply(lambda x: x.hour * 60 + x.minute)

df_smoothed_profile

In [ ]:
df_data.index.date[0]

In [ ]:
from scipy.interpolate import interp1d
from datetime import datetime, timedelta

non_zero_profile = df_smoothed_profile[df_smoothed_profile["P_max"] > 0]
T_start_profile = non_zero_profile["time_of_day"].min()
T_end_profile = non_zero_profile["time_of_day"].max()
print("T_start_profile: ", T_start_profile)
print("T_end_profile: ", T_end_profile)
print(" ")

#daylength profile
T_start_profile_min = T_start_profile.hour * 60 + T_start_profile.minute
T_end_profile_min = T_end_profile.hour * 60 + T_end_profile.minute
daylength_profile = T_end_profile_min - T_start_profile_min + 1 #inclusive
print("daylength profile: ", daylength_profile)

profile_data = df_smoothed_profile.iloc[T_start_profile_min // 15:T_end_profile_min // 15 + 1]
print(profile_data)

def adjust_pmax_2(day):
    print("day")
    print(day)
    date = day["index"]
    
    #start of day according to sunrise and sunset per definition
    
    #T_start_day = day["T_start"].iloc[0] #convert the value in the column to just the value
    #T_end_day = day["T_end"].iloc[0]
    print("date: ", date)
    
    #start of day according to sunrise and sunset per first non zero value
    non_zero_profile_day = df_data[(df_data.index.date == date.iloc[0]) & (df_data["mean_actualPowerTot_W_inverter"] > 0)]
    if not non_zero_profile_day.empty:
        T_start_day = non_zero_profile_day.index.min()
        T_end_day = non_zero_profile_day.index.max()
    else: #take the one per definition
        T_start_day = day["T_start"].iloc[0] #convert the value in the column to just the value
        T_end_day = day["T_end"].iloc[0]
    
    print("T_start_day: ", T_start_day)
    print("T_end_day: ", T_end_day)
    
    #calculate daylength day
    T_start_day_min = T_start_day.hour * 60 + T_start_day.minute
    T_end_day_min = T_end_day.hour * 60 + T_end_day.minute
    daylength_day = T_end_day_min - T_start_day_min + 1 #inclusive
    print("daylength day: ", daylength_day)
    
    #factor: need to put daylength_profile amount of values in daylength_day spaces
    ratio = daylength_day/daylength_profile
    
    #generate original and new time indices
    original_time = np.linspace(0, daylength_day, daylength_profile // 15 + 1)
    new_time = np.linspace(0, daylength_day, daylength_day // 15 + 1)

    
    #interpolate the profile data
    interpolator = interp1d(original_time, profile_data["P_max"], kind='linear')
    adjusted_profile = interpolator(new_time)
    
    #we have new P_max values --> new dataframe    
    zero_padding_before = np.zeros(max(0, T_start_day_min // 15))
    zero_padding_after = np.zeros(max(0, df_smoothed_profile.shape[0] - (T_end_day_min // 15)))


    print("zero_padding_before: ", len(zero_padding_before))
    print("zero_padding_after: ", len(zero_padding_after))
    
    final_profile = np.concatenate([zero_padding_before, adjusted_profile, zero_padding_after])
    final_time = np.linspace(0, daylength_day, len(final_profile))
    
    adjusted_df = pd.DataFrame({
        #"time": final_time,
        "P_max": final_profile
    })
    print(adjusted_df)
    
    return adjusted_df
    
full_adjusted_df = pd.DataFrame()

for date in daily_sun_times["index"]:
    adjusted_profile_df = adjust_pmax_2(daily_sun_times[daily_sun_times["index"] == date])
    
    adjusted_profile_df["hours"] = (adjusted_profile_df.index * 15) // 60
    adjusted_profile_df["minutes"] = (adjusted_profile_df.index * 15) % 60

    
    date_datetime = datetime.strptime(str(date), "%Y-%m-%d")  
    
    adjusted_profile_df["time"] = adjusted_profile_df.apply(
        lambda row: date_datetime + timedelta(hours=row["hours"], minutes=row["minutes"]), axis=1
    )
    adjusted_profile_df = adjusted_profile_df.drop(columns=["hours", "minutes"])
    
    full_adjusted_df = pd.concat([full_adjusted_df, adjusted_profile_df], ignore_index=True)
    #break
    
full_adjusted_df.set_index("time", inplace=True)

print(full_adjusted_df.head())

step2_1 = go.Figure()

step2_1.add_trace(go.Scatter(x=adjusted_profile_df.index, y=adjusted_profile_df["P_max"], mode='lines', name='adjusted'))
#step2_1.add_trace(go.Scatter(x=df_smoothed_profile.index, y=df_smoothed_profile["P_max"], mode='lines', name='step1'))

#update layout for better visualization
step2_1.update_layout(
    title='max profile step2_1',
    xaxis_title='Time',
    yaxis_title='Mean Actual Power (W)',
    xaxis_rangeslider_visible=True
)

step2_1.show()

In [ ]:
full_adjusted_df

In [ ]:
#transformation 2
#section_transformed_df = full_adjusted_df[full_adjusted_df.index <= '2021-08-11 00:00:00']

section_transformed_df = full_adjusted_df

step2_2 = go.Figure()

step2_2.add_trace(go.Scatter(x=section_transformed_df.index, y=section_transformed_df["P_max"], mode='lines', name='transformation 2'))
#step2_1.add_trace(go.Scatter(x=df_smoothed_profile.index, y=df_smoothed_profile["P_max"], mode='lines', name='step1'))

#update layout for better visualization
step2_2.update_layout(
    title='max profile transformation 2',
    xaxis_title='Time',
    yaxis_title='Mean Actual Power (W)',
    xaxis_rangeslider_visible=True
)

step2_2.show()

In [ ]:
import pvlib
import elevation
from geopy.geocoders import Nominatim

# Latitude and longitude of the location
latitude = df_households.loc[df_households["id"] == baseId, "latitude"].iloc[0]
longitude = df_households.loc[df_households["id"] == baseId, "longitude"].iloc[0]

print(pvlib.location.lookup_altitude(latitude, longitude))
#for now default to 10m

#altitude = 10

altitude = 8

In [ ]:
# transformation 3: GHI
# use clearsky ineichen

location = pvlib.location.Location(latitude, longitude, altitude=altitude)

#convert the index to a pandas datetime index with UTC timezone
times = pd.DatetimeIndex(full_adjusted_df.index, tz='UTC')

#get clearsky data for all timestamps at once
clearsky_data = location.get_clearsky(times)

#extract the GHI values and assign them to the dataframe in one go
full_adjusted_df['clearsky_ghi'] = clearsky_data['ghi'].values

full_adjusted_df

In [ ]:
max_ghi_overall = full_adjusted_df['clearsky_ghi'].max()

full_adjusted_df['date'] = full_adjusted_df.index.date
daily_max_ghi = full_adjusted_df.groupby('date')['clearsky_ghi'].max()

daily_max_ghi_ratio = daily_max_ghi / max_ghi_overall

full_adjusted_df['daily_max_ghi_ratio'] = full_adjusted_df['date'].map(daily_max_ghi_ratio)

full_adjusted_df['adjusted_P_max'] = full_adjusted_df['P_max'] * full_adjusted_df['daily_max_ghi_ratio']

In [ ]:
full_adjusted_df = full_adjusted_df.drop(columns = ["clearsky_ghi", "date"])
full_adjusted_df

In [ ]:
# transformation 3
#section_transformed_df = full_adjusted_df[full_adjusted_df.index <= '2021-08-11 00:00:00']

section_transformed_df = full_adjusted_df


step2_3 = go.Figure()

step2_3.add_trace(go.Scatter(x=section_transformed_df.index, y=section_transformed_df["P_max"], mode='lines', name='transformation 2'))
step2_3.add_trace(go.Scatter(x=section_transformed_df.index, y=section_transformed_df["adjusted_P_max"], mode='lines', name='transformation 3'))

#update layout for better visualization
step2_3.update_layout(
    title='max profile transformation 3',
    xaxis_title='Time',
    yaxis_title='Mean Actual Power (W)',
    xaxis_rangeslider_visible=True
)

step2_3.show()

In [ ]:
#try to fix that ideal profile is always bigger than actual profile

df_data['date'] = df_data.index.date
full_adjusted_df['date'] = full_adjusted_df.index.date

#create a copy of adjusted_P_max to modify
full_adjusted_df['scaled_adjusted_P_max'] = full_adjusted_df['adjusted_P_max']

#get all unique dates
unique_dates = df_data['date'].unique()

for day in unique_dates:
    #get data for the day
    actual_day = df_data[df_data['date'] == day]
    adjusted_day = full_adjusted_df[full_adjusted_df['date'] == day]

    #get max values for that day
    max_actual = actual_day['mean_actualPowerTot_W_inverter'].max()
    max_adjusted = adjusted_day['adjusted_P_max'].max()

    if max_adjusted < max_actual:
        scale_factor = max_actual / max_adjusted
        print(f"Scaling {day}: {scale_factor:.2f}")
        
        #apply scaling for that day's adjusted profile
        day_mask = full_adjusted_df['date'] == day
        full_adjusted_df.loc[day_mask, 'scaled_adjusted_P_max'] = full_adjusted_df.loc[day_mask, 'scaled_adjusted_P_max'] * scale_factor


full_adjusted_df.drop(columns=['date'], inplace=True)
df_data.drop(columns=['date'], inplace=True)


In [ ]:
full_adjusted_df

In [ ]:
# all transformations done
#now normalize data with normalization profile

df_data.index = df_data.index.tz_localize(None)  # Remove any timezone if present
full_adjusted_df.index = full_adjusted_df.index.tz_localize(None)  # Remove any timezone if present

#merge the two DataFrames on the 'time' index
df_merged = df_data.merge(full_adjusted_df[['scaled_adjusted_P_max']], left_index=True, right_index=True, how='left')

df_merged.index.name = '_time'
#lose some data from full_adjusted_df near end since the df_data doesn't have full last day

#check for missing values (NaN) in adjusted_P_max
if df_merged['scaled_adjusted_P_max'].isna().any():
    print("Warning: Some values are missing in the normalization profile.")
    
#normalize the 'mean_actualPowerTot_W_inverter' column by dividing by the 'adjusted_P_max' column
#df_merged['normalized_value'] = df_merged['mean_actualPowerTot_W_inverter'] / df_merged['adjusted_P_max']
df_merged['normalized_value'] = np.where(
    (df_merged['mean_actualPowerTot_W_inverter'] == 0) | (df_merged['scaled_adjusted_P_max'] == 0),
    np.nan,  #set to NaN if either value is 0
    df_merged['mean_actualPowerTot_W_inverter'] / df_merged['scaled_adjusted_P_max']  # Perform division otherwise
)

df_merged

In [ ]:
df_merged[df_merged['scaled_adjusted_P_max']>0]

In [ ]:
#fix the nan at night
df_merged['normalized_value'] = df_merged['normalized_value'].clip(upper=1)

#assign a 'night'-variable for NaN values
df_merged['night'] = df_merged['normalized_value'].isna().astype(int)

df_merged_transform = df_merged.copy(deep = True)

#initialize the first night
firstNight = True
firstSun = True
sumValue = 0
count = 0
average = 0
first_sunrise = 0

for _, row in df_merged_transform.iterrows():
    if (row['night'] == 1) & (firstNight):
        #skip first night
        continue
    firstNight = False
    if firstSun:
        first_sunrise = row.index
        firstSun = False
    #start of first day
    if row['night'] == 0:
        sumValue += row['normalized_value']
        count += 1
        continue
    else:
        #summed all day values
        average = sumValue / count
        print('average: ', average)
        break
        
firstNight = True
for idx, row in df_merged_transform.iterrows():
    if row['night'] == 1 & firstNight:
        df_merged_transform.loc[idx, 'normalized_value'] = average
        continue
    firstNight = False
    break

#for the remaining nights
sum_prev_night = []
average_prev_day = 0

counter = 0

for idx, row in df_merged_transform.iterrows():
    if(pd.notna(row['normalized_value'])):
        sum_prev_night.append(row['normalized_value'])
    else:
        #reset sum array after average calculation, once
        if sum_prev_night:
            average_prev_day = sum(sum_prev_night)/ len(sum_prev_night)
            sum_prev_night = []
            
        df_merged_transform.at[idx, 'normalized_value'] = average_prev_day
    
    if counter % 1000 == 0:
        print(f"Processing row {counter}")
    counter += 1

#df_merged_transform_rest = df_merged_transform
print(first_sunrise)

df_merged_transform

In [ ]:
#transform done:
df_merged = df_merged_transform

In [ ]:
df_merged

In [ ]:
# normalized dataset
#section_normalized_df = df_merged[df_merged.index <= '2021-08-11 00:00:00']
section_normalized_df = df_merged

step3 = go.Figure()

#step3.add_trace(go.Scatter(x=section_normalized_df.index, y=section_normalized_df["adjusted_P_max"]/1000, mode='lines', name='transformation 3 original'))
step3.add_trace(go.Scatter(x=section_normalized_df.index, y=section_normalized_df["scaled_adjusted_P_max"]/1000, mode='lines', name='clear-sky profile'))
#step3.add_trace(go.Scatter(x=section_normalized_df.index, y=section_normalized_df["normalized_value"], mode='lines', name='normalized'))
#step3.add_trace(go.Scatter(x=section_normalized_df.index, y=section_normalized_df["mean_actualPowerTot_W_inverter"]/1000, mode='lines', name='actual production'))


step3.update_layout(
    title='normalized profile',
    xaxis_title='Time',
    yaxis_title='Mean Actual Power (kW)',
    showlegend=True,
    #xaxis_rangeslider_visible=True,
    margin=dict(t=150),  # Increase top margin to fit legend and title
    legend=dict(
        orientation="h",  # horizontal layout
        y=1,           # place it above the plot area
        x=0.5,
        xanchor='center',
        yanchor='bottom'
    ),
    plot_bgcolor='white',
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        range=['2022-05-05', '2022-05-8']
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgray'
    )
)


step3.show()

In [ ]:
#temporary
# normalized dataset
#section_normalized_df = df_merged[df_merged.index <= '2021-08-11 00:00:00']
section_normalized_df = df_merged

step3 = go.Figure()

#step3.add_trace(go.Scatter(x=section_normalized_df.index, y=section_normalized_df["adjusted_P_max"]/1000, mode='lines', name='transformation 3 original'))
step3.add_trace(go.Scatter(x=section_normalized_df.index, y=section_normalized_df["scaled_adjusted_P_max"]/1000, mode='lines', name='clear-sky profile'))
#step3.add_trace(go.Scatter(x=section_normalized_df.index, y=section_normalized_df["normalized_value"], mode='lines', name='normalized'))
#step3.add_trace(go.Scatter(x=section_normalized_df.index, y=section_normalized_df["mean_actualPowerTot_W_inverter"]/1000, mode='lines', name='actual production'))


step3.update_layout(
    title='clear-sky profile',
    xaxis_title='Time',
    yaxis_title='Mean Actual Power (kW)',
    xaxis_rangeslider_visible=True,
    margin=dict(t=150),  # Increase top margin to fit legend and title
    legend=dict(
        orientation="h",  # horizontal layout
        y=1,           # place it above the plot area
        x=0.5,
        xanchor='center',
        yanchor='bottom'
    ),
    showlegend=True,
)


# Show the plot
step3.show()

In [ ]:
df_merged['adjusted_P_max'] = df_merged['scaled_adjusted_P_max']

file_path = r"C:\Users\samr0\OneDrive - KU Leuven\Documents\!School\master\Thesis\data\inverter_power_data_ee9f3d22_normalised_15min.csv"

df_merged.to_csv(file_path)

In [ ]:
#reconstruct
df_test = df_merged.copy()
df_test["denormalized_value"] = df_test["normalized_value"]*df_test["scaled_adjusted_P_max"]
df_test

In [ ]:
# denormalized dataset
denormalized = go.Figure()

denormalized.add_trace(go.Scatter(x=df_test.index, y=df_test["scaled_adjusted_P_max"]/1000, mode='lines', name='transformation 3 scaled'))
denormalized.add_trace(go.Scatter(x=df_test.index, y=df_test["adjusted_P_max"]/1000, mode='lines', name='transformation 3'))
denormalized.add_trace(go.Scatter(x=df_test.index, y=df_test["denormalized_value"]/1000, mode='lines', name='denormalized'))
denormalized.add_trace(go.Scatter(x=df_test.index, y=df_test["normalized_value"], mode='lines', name='normalized'))
denormalized.add_trace(go.Scatter(x=df_test.index, y=df_test["mean_actualPowerTot_W_inverter"]/1000, mode='lines', name='mean_actualPowerTot_W_inverter'))


# Update layout for better visualization
denormalized.update_layout(
    title='denormalized',
    xaxis_title='Time',
    yaxis_title='Mean Actual Power (kW)',
    xaxis_rangeslider_visible=True
)

# Show the plot
denormalized.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
r2 = r2_score(df_test["mean_actualPowerTot_W_inverter"], df_test["denormalized_value"])
print("R²-score: ", r2)